# Climat change assistant

This notebook shows how to set up a Langchain LECL chain to answer user questions about climate in their area and provide assistance with related documentation from the US Federal Emergency Management Agency (FEMA).

# Setup

1. Get a [Probable futures API Key](https://docs.probablefutures.org/api-access/)
2. Register an account on Langsmith and get an API key
3. See [README](./README) to set up a conda environment and `.env` file

In [242]:
import nest_asyncio
import os
import openai
from dotenv import load_dotenv
import json
import os
import time
import pandas as pd
import openpyxl
import shutil
import requests

from langchain_community.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
#from langchain_chroma import Chroma
from langchain.vectorstores.chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import PromptTemplate
from langchain import callbacks

from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)
from langchain.tools import Tool
from langsmith import Client


import assistant_tools as at

nest_asyncio.apply()
pd.set_option("display.max_colwidth", None)

# Load environment variables from .env file
load_dotenv()

# Initialize the OpenAI API client
openai.api_key = os.getenv("OPENAI_API_KEY")

DATA_DIR = os.getenv("DATA_DIR")
DB_DIR = os.getenv("DB_DIR")
DB_COLLECTION_NAME = os.getenv("DB_COLLECTION_NAME")
CHAT_MODEL = os.getenv("OPENAI_CHAT_MODEL")
EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
DEEPEVAL_API_KEY = os.getenv("DEEPEVAL_API_KEY")
PDF_DIR = f"{DATA_DIR}/pdfs"

os.makedirs(PDF_DIR, exist_ok=True)
os.makedirs(DB_DIR, exist_ok=True)

embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

# LangSmith Client
client = Client()

# Analysis

## Indexing FEMA Disaster preparedness documents

### Get FEMA PDF documents

In [5]:
df = pd.read_csv(f"{DATA_DIR}/fema_docs.csv")
display(df)

,Source,URL,Extra instructions,Document
0,FEMA,https://www.fema.gov/emergency-managers/risk-m...,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/2020-...
1,FEMA,https://www.fema.gov/emergency-managers/risk-m...,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/2020-...
2,FEMA,https://www.fema.gov/emergency-managers/risk-m...,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/2020-...
3,FEMA,https://www.fema.gov/emergency-managers/risk-m...,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/2020-...
4,FEMA,https://www.fema.gov/emergency-managers/risk-m...,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/2020-...
5,FEMA,https://www.fema.gov/emergency-managers/risk-m...,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/2020-...
6,FEMA,https://www.fema.gov/emergency-managers/risk-m...,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/docum...
7,FEMA,https://www.fema.gov/emergency-managers/risk-m...,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/2020-...
8,FEMA,https://www.fema.gov/emergency-managers/indivi...,NaN,https://www.fema.gov/sites/default/files/2020-...
9,FEMA,https://www.fema.gov/emergency-managers/indivi...,NaN,https://www.fema.gov/sites/default/files/2020-...


### Build Vector Database

First we will build a FEMA RAG chain for asnwering questions about preparing for disasters, using FEMA PDFs.

In [8]:
# Download all documents as defined in 'Documents' column
for doc_url in df["Document"]:
    print(f"Downloading {doc_url}")
    response = requests.get(doc_url)
    with open(f"{PDF_DIR}/{doc_url.split('/')[-1]}", "wb") as f:
        f.write(response.content)


### Index documents

We will use a very simple parser and chunking methodology to ingest documents for this demo.

In [2]:
# Load the PDFs
docs = []
for pdf_file in os.listdir(PDF_DIR):
    if not pdf_file.endswith(".pdf"):
        continue
    print(f"Loading PDF: {pdf_file}")
    file_path = f"{PDF_DIR}/{pdf_file}"
    loader = PyPDFLoader(file_path)
    docs = docs + loader.load()
    print(f"Loaded {len(docs)} documents")

print(len(docs))

Loading PDF: fema_scenario_10_power_outage_answer_key_01102020.pdf
Loaded 2 documents
Loading PDF: fema_scenario_7-shelter_in_place_TTX_answer_key_01102020.pdf
Loaded 5 documents
Loading PDF: ready_12-ways-to-prepare_postcard.pdf
Loaded 7 documents
Loading PDF: fema_safeguard-critical-documents-and-valuables.pdf
Loaded 10 documents
Loading PDF: ready_document-and-insure-your-property.pdf
Loaded 16 documents
Loading PDF: fema_scenario_1-active_shooter-01102020.pdf
Loaded 18 documents
Loading PDF: fema_protect-your-property_wildfire.pdf
Loaded 26 documents
Loading PDF: fema_scenario_4-hurricane-01102020.pdf
Loaded 27 documents
Loading PDF: fema_scenario_10_power_outage_01102020.pdf
Loaded 28 documents
Loading PDF: fema_scenario_4_hurricane_flood_TTX_answer_key-01102020.pdf
Loaded 30 documents
Loading PDF: fema_scenario_11_winter_storm_01102020.pdf
Loaded 32 documents
Loading PDF: fema_protect-your-property_severe-wind.pdf
Loaded 44 documents
Loading PDF: fema_protect-your-property-storm-

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [4]:
if os.path.exists(DB_DIR):
    print("Removing existing DB directory")
    shutil.rmtree(DB_DIR)

print(f"Creating vector store and saving to {DB_DIR}")
vector_store = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=DB_DIR,
    collection_name=DB_COLLECTION_NAME
)

Removing existing DB directory
Creating vector store and saving to ./db


### Base retriever

In [268]:
vector_store = Chroma(persist_directory=DB_DIR, embedding_function=embeddings, collection_name=DB_COLLECTION_NAME)
retriever = vector_store.as_retriever(search_kwargs={"k": 15})

In [269]:
results = retriever.invoke("How can I prepare my house for a flood?")
for doc in results[0:3]:
    print("=====================================")
    print(doc.metadata)
    print(doc.page_content)

{'page': 3, 'source': './data/pdfs/fema_protect-your-property-storm-surge.pdf'}
your agent to  
get coverage.
 
 PREPARE OR  
UPDATE A LIST OF 
 YOUR HOME’S 
CONTENTSDocument your belongings. This will give you peace of mind 
and help with the insurance process if you need to file a claim. Consider documenting your home’s contents visually. You can either take photos of high-value items or walk through your home and videotape each room’s belongings.
STORE  
VALUABLESStore valuables and important documents above the BFE (preferably on an upper floor). Place them in waterproof or 
water-resistant containers. Also, make copies and store them 
online or offsite.
ELEVATE 
APPLIANCES AND 
UTILITIES ABOVE 
THE BFEKeep appliances and utilities such as water heaters, washers, dryers, and 
electric panels on higher 
floors. It can prevent them from getting damaged or ruined by flood water.
Talk to your floodplain 
manager about how high to elevate your utilities. Many coastal
{'page': 3, 'source

/Users/matthewharris/opt/miniconda3/envs/climate-bot/lib/python3.11/site-packages/pydantic/main.py:1111: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


Output a few results, the text seems reasonable for the query.

### Build FEMA RAG Chain

In [270]:
def output_results(results):
    """
    Output the results of a question-answering task.

    Args:
        results (dict): The results of the question-answering task.

    Returns:
        None
    """
    print("Question:")
    print(results["query"])
    print("Answer:")
    print(results["result"])
    print("\nContext:")
    for doc in results["source_documents"]:
        print(doc.page_content)
        print("\n\n========\n")
    print("\n")

system_prompt = (
    "You are an assistant helping people prepare for disasters."
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. "
    "You ONLY answer using the context provided. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}\n{context}"),
    ]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_llm = ChatOpenAI(model=CHAT_MODEL, temperature=0.0)

# # This returns the context and the response, but breaks tracing on deepeval
# I changed to 'output' so deepeval tracing works
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | rag_llm
    | StrOutputParser()
)
fema_rag_chain = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(output=rag_chain_from_docs)



Let's test it ...

In [271]:
query = "How can I prepare for floods??"
results = fema_rag_chain.invoke(query)
print(results["output"],"\n\n")
for doc in results["context"]:
    print(doc.metadata)


/Users/matthewharris/opt/miniconda3/envs/climate-bot/lib/python3.11/site-packages/pydantic/main.py:1111: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


To prepare for floods, you can take several steps both inside and outside your home:

### Inside Your Home:
1. **Have a Plan**: Create an emergency plan for your family and practice it regularly. When a storm is approaching, evacuate and move your car to higher ground. Visit [Ready.gov/plan](https://www.ready.gov/plan) for more information.
2. **Get Flood Insurance**: Most homeowners insurance policies don’t cover flood damage. Purchase flood insurance for your home and its contents, even if you do not live in a high-risk flood zone. Visit [FloodSmart.gov](https://www.floodsmart.gov) or contact your insurance agent to get coverage.
3. **Prepare or Update a List of Your Home’s Contents**: Document your belongings to help with the insurance process if you need to file a claim. Take photos of high-value items or do a video walkthrough of your home.
4. **Store Valuables**: Store valuables and important documents above the Base Flood Elevation (BFE), preferably on an upper floor, in waterpr

Great! That works well. Obviously a production RAG system may need more sophisticated parsing and chunking strategies, as well as refinement of the summarization prompt to ensure no key information is ommited, but this at least provides the correct content.

## Getting data from probable futures for a location

The [Probable futures](https://probablefutures.org/) API is a graph API, so I have created a few helper functions in [./assistant_tools.py](./assistant_tools.py) to authenticate and extract information for a location.

Let's do a quick test ...

In [272]:
resp = at.get_pf_data(address="Burlington", country="United States", warming_scenario="1.5")
print(resp)


        mutation {
            getDatasetStatistics(input: { 
        country: "United States"
        address: "Burlington"
                         warmingScenario: "1.5" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    
{
  "data": {
    "getDatasetStatistics": {
      "datasetStatisticsResponses": [
        {
          "datasetId": 40107,
          "midValue": "0.0",
          "name": "Days above 45\u00b0C (113\u00b0F)",
          "unit": "days",
          "warmingScenario": "1.5",
          "latitude": 44.4,
          "longitude": -73.2,
          "info": {}
        },
        {
          "datasetId": 40206,
          "midValue": "20.0",
          "name": "10 hottest nights",
          "unit":

### Building a chain to classify climate predictions

Let's build a chain to classify the results from Probable futures. Basically, we want to identify any areas where a person might want toi prepare, and use that to get corresponding documentation from FEMA.

For this we will create a simple 

In [273]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def get_pf_info(query):
    """
    Get information about a location's predicted future climate.
    
    Args:
        query (str): The location to query.

    Returns:
        dict: The predicted future climate information.
    """
    resp = at.get_pf_data(address=query, country="United States", warming_scenario="1.5")
    return resp

get_pf_info.run({"query": "Burlington vermont"})


        mutation {
            getDatasetStatistics(input: { 
        country: "United States"
        address: "Burlington vermont"
                         warmingScenario: "1.5" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    


'{\n  "data": {\n    "getDatasetStatistics": {\n      "datasetStatisticsResponses": [\n        {\n          "datasetId": 40107,\n          "midValue": "0.0",\n          "name": "Days above 45\\u00b0C (113\\u00b0F)",\n          "unit": "days",\n          "warmingScenario": "1.5",\n          "latitude": 44.4,\n          "longitude": -73.2,\n          "info": {}\n        },\n        {\n          "datasetId": 40206,\n          "midValue": "20.0",\n          "name": "10 hottest nights",\n          "unit": "\\u00b0C",\n          "warmingScenario": "1.5",\n          "latitude": 44.4,\n          "longitude": -73.2,\n          "info": {}\n        },\n        {\n          "datasetId": 40207,\n          "midValue": "-5.0",\n          "name": "Average winter temperature",\n          "unit": "\\u00b0C",\n          "warmingScenario": "1.5",\n          "latitude": 44.4,\n          "longitude": -73.2,\n          "info": {}\n        },\n        {\n          "datasetId": 40601,\n          "midValue": "6

In [260]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [
    get_pf_info,  
]


system_prompt = (
"""
================================ System Message ================================

You are a an AI assistant that analyzes output from a climate change AI.
You assess whether the data indicates extreme changes in climate.
Only focus on changes due to climate change, do not focus on existing conditions
Based on the climate change difference data, types of events people should prepare for, be explicit: floods, drought, fire etc
Be conservative, only advise on the most extreme events:
   - Less than 20 extra of an event per year is not extreme
   - A percentage likelihood increase less than 30% is not extreme
If you don't know the answer, say that you don't know.
You ONLY answer using the context provided.


============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}
""")


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

llm = ChatOpenAI(model=CHAT_MODEL, temperature=0.0)
agent = create_tool_calling_agent(llm, tools, prompt)
climate_api_agent = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [274]:
api_result = climate_api_agent.invoke(
    {
        "input": "How will climate change affect Burlington vermont"
    }
)
print(api_result["output"])



> Entering new AgentExecutor chain...

Invoking: `get_pf_info` with `{'query': 'Burlington, Vermont'}`



        mutation {
            getDatasetStatistics(input: { 
        country: "United States"
        address: "Burlington, Vermont"
                         warmingScenario: "1.5" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    
{
  "data": {
    "getDatasetStatistics": {
      "datasetStatisticsResponses": [
        {
          "datasetId": 40107,
          "midValue": "0.0",
          "name": "Days above 45\u00b0C (113\u00b0F)",
          "unit": "days",
          "warmingScenario": "1.5",
          "latitude": 44.4,
          "longitude": -73.2,
          "info": {}
        },
        {


In [279]:

system_prompt = (
    "You are an assistant telling people how to prepare for types of events related to climate change"
    "Use the following information to generate hypothetical user questions about climate change."
    "If the information mentions twice as many storms and high precipitation, the question might be: How can I prepare for floods?"
    "If the information indicates a large increase in wildfires (> 20 days a year), then the question might be: What are the risks of wildfires in my area?"
    "If the information indicates increases in heat, the question might be: Is there a way I can prepare for extreme heat?"
    "Do not generate questions for every scenario, only those where the information says there will be a significant increase"
    "Anything with a small increase or no increase, do not generate a question for"
    "Don't generate questions that ask about risks, only about how to prepare for events"
    "Questions should always be about how to prepare for events, not about the likelihood of the event"
    "ONLY output the questions"
    "You ONLY answer using the context provided. "
    "\n\n"
    "{input}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

climate_advice_llm = ChatOpenAI(model=CHAT_MODEL, temperature=0.0)

advice_chain = (
    RunnablePassthrough.assign(input=(lambda x: x["api_agent_result"]))
    | prompt
    | climate_advice_llm
    | RunnableLambda(lambda x: {"output": x.content})
)

advice = advice_chain.invoke({
    "api_agent_result": api_result["output"]
})
print(advice)

{'output': 'How can I prepare for floods?\n\nHow can I prepare for wildfires?'}


We have an llm step to take the user input and call the tool to get predicted climate change. Now let's add an llm to generate a documentation query.

In [280]:
from langchain_core.runnables.passthrough import RunnablePick

# It's a bit cumbersome, but we will pass through variables from each stage for potential use in the UI
full_chain = RunnableParallel({
    "user_question": RunnablePick("input"),
    "api_agent_result": climate_api_agent | RunnablePick("output"),             # API Agent to get predictions
}) | RunnableParallel({
    "user_question": RunnablePick("user_question"),
    "api_agent_result": RunnablePick("api_agent_result"),
    "advice_chain_result": advice_chain | RunnablePick("output"),               # Advice chain to generate RAG question
}) | RunnableParallel({
   "user_question": RunnablePick("user_question"),
   "api_agent_result": RunnablePick("api_agent_result"),
   "advice_chain_result": RunnablePick("advice_chain_result"),
   "fema_rag_chain_result": RunnablePick("api_agent_result") | fema_rag_chain,  # FEMA RAG chain to answer RAG question
}) 

In [281]:
result = full_chain.invoke({
    "input": "How will climate change affect Reno Nevada"
})




> Entering new AgentExecutor chain...

Invoking: `get_pf_info` with `{'query': 'Reno, Nevada'}`



        mutation {
            getDatasetStatistics(input: { 
        country: "United States"
        address: "Reno, Nevada"
                         warmingScenario: "1.5" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    
{
  "data": {
    "getDatasetStatistics": {
      "datasetStatisticsResponses": [
        {
          "datasetId": 40107,
          "midValue": "0.0",
          "name": "Days above 45\u00b0C (113\u00b0F)",
          "unit": "days",
          "warmingScenario": "1.5",
          "latitude": 39.6,
          "longitude": -119.8,
          "info": {}
        },
        {
          "da

/Users/matthewharris/opt/miniconda3/envs/climate-bot/lib/python3.11/site-packages/pydantic/main.py:1111: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


In [282]:
for key in ["user_question", "api_agent_result", "advice_chain_result"]:
    print(f"\n\n{key}:\n\n {result[key]}")

print("\n\nFEMA RAG Chain Result:\n\n")
print(result["fema_rag_chain_result"]["output"])
print("\n\nFEMA Further Reading:\n\n")
for doc in result["fema_rag_chain_result"]["context"]:
    print(doc.metadata)




user_question:

 How will climate change affect Reno Nevada


api_agent_result:

 Based on the climate change data for Reno, Nevada, here are the key impacts:

1. **Increase in Dry Hot Days**: There will be an additional 19 dry hot days per year. This could lead to increased risk of heat-related illnesses and stress on water resources.

2. **Likelihood of Drought**: The likelihood of experiencing a year-plus drought is 43%. This is a significant increase and suggests that Reno should prepare for prolonged periods of water scarcity.

3. **Extreme Drought**: There is a 17% likelihood of experiencing a year-plus extreme drought. This further emphasizes the need for water conservation and management strategies.

4. **Change in Precipitation for Extreme Storms**: The frequency of "1-in-100-year" storms will increase by 2 times, and the precipitation during these storms will increase by 8 mm. This indicates a higher risk of flooding during extreme weather events.

5. **Decrease in Snowy Da

## Tracing

Here we will set up tracing in LangSmith

In [283]:
with callbacks.collect_runs() as cb:
    result = full_chain.invoke({
        "input": "How will climate change affect Burlington vermont"
    })
    run_id = cb.traced_runs[0].id
    print(f"Run ID: {run_id}")
    print(f"Result: {print(result['fema_rag_chain_result']['output'])}")



> Entering new AgentExecutor chain...

Invoking: `get_pf_info` with `{'query': 'Burlington, Vermont'}`



        mutation {
            getDatasetStatistics(input: { 
        country: "United States"
        address: "Burlington, Vermont"
                         warmingScenario: "1.5" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    
{
  "data": {
    "getDatasetStatistics": {
      "datasetStatisticsResponses": [
        {
          "datasetId": 40107,
          "midValue": "0.0",
          "name": "Days above 45\u00b0C (113\u00b0F)",
          "unit": "days",
          "warmingScenario": "1.5",
          "latitude": 44.4,
          "longitude": -73.2,
          "info": {}
        },
        {


/Users/matthewharris/opt/miniconda3/envs/climate-bot/lib/python3.11/site-packages/pydantic/main.py:1111: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


Run ID: 30425679-8e33-486c-b557-6c58069c6d74
### Key Climate Change Impacts for Burlington, Vermont:

1. **Increase in Total Annual Precipitation**: Projected increase of 64 mm in total annual precipitation, leading to more frequent and intense rainfall events.
2. **Increase in Frequency of "1-in-100-year" Storms**: These extreme storms are expected to become twice as frequent, indicating a higher likelihood of severe weather events.
3. **Increase in Precipitation During "1-in-100-year" Storms**: Projected increase of 13 mm in precipitation during these extreme storms, exacerbating flooding risks.
4. **Decrease in Snowy Days**: Projected decrease of 10 snowy days per year, impacting winter sports and water resources that rely on snowmelt.
5. **Increase in Dry Hot Days**: Projected increase of 8 dry hot days per year, contributing to drought conditions and increasing wildfire risks.
6. **Increase in Wildfire Danger Days**: Projected increase of 9 wildfire danger days per year, indicatin

In [284]:
with callbacks.collect_runs() as cb:
    result = full_chain.invoke({
        "input": "How will climate change affect Burlington vermont"
    })
    run_id = cb.traced_runs[0].id
    run = client.read_run(run_id)
    print(f"Run URL: {run.url}")
    print(f"Run ID: {run_id}")




> Entering new AgentExecutor chain...

Invoking: `get_pf_info` with `{'query': 'Burlington, Vermont'}`



        mutation {
            getDatasetStatistics(input: { 
        country: "United States"
        address: "Burlington, Vermont"
                         warmingScenario: "1.5" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    
{
  "data": {
    "getDatasetStatistics": {
      "datasetStatisticsResponses": [
        {
          "datasetId": 40107,
          "midValue": "0.0",
          "name": "Days above 45\u00b0C (113\u00b0F)",
          "unit": "days",
          "warmingScenario": "1.5",
          "latitude": 44.4,
          "longitude": -73.2,
          "info": {}
        },
        {


/Users/matthewharris/opt/miniconda3/envs/climate-bot/lib/python3.11/site-packages/pydantic/main.py:1111: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


Run URL: https://smith.langchain.com/o/c25b80aa-0e39-544a-8c93-8567c04637a7/projects/p/4eee31d8-2d67-4d00-a230-8d60aecc4f10/r/1425bee8-9fc2-4f93-adf8-d05026203367?trace_id=1425bee8-9fc2-4f93-adf8-d05026203367&start_time=2024-09-15T23:49:49.453029
Run ID: 1425bee8-9fc2-4f93-adf8-d05026203367


## Evaluation

First let's generate an evaluation test set.

Assuming some events may occur more fequently in some locations, we can map to questions ...

Florida - Hurricanes: What can I do to prepare for hurricanes?
California--Wildfires: How can I protect my home from Wildfires?
Arizona - Drought: What should my family do to prepare for drought?

In [285]:
seed_data = [
    "How will climate change in Miami",
    "I live in California, how will climate change affect me?",
    "My home doesn't have its own water supply, how can I prepare for drought in Arizona?",
]

eval_data = []
for question in seed_data:
    result = full_chain.invoke({
        "input": question
    })

    docs = []
    for d in result["fema_rag_chain_result"]["context"]:
        docs.append(d.page_content)

    eval_data.append(
        {
            "query": question,
            "ground_truths": result["fema_rag_chain_result"]["output"],
            "context": docs,
        }
    )

df = pd.DataFrame(eval_data)
display(df)



> Entering new AgentExecutor chain...

Invoking: `get_pf_info` with `{'query': 'Miami'}`



        mutation {
            getDatasetStatistics(input: { 
        country: "United States"
        address: "Miami"
                         warmingScenario: "1.5" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    
{
  "data": {
    "getDatasetStatistics": {
      "datasetStatisticsResponses": [
        {
          "datasetId": 40107,
          "midValue": "0.0",
          "name": "Days above 45\u00b0C (113\u00b0F)",
          "unit": "days",
          "warmingScenario": "1.5",
          "latitude": 25.8,
          "longitude": -80.2,
          "info": {}
        },
        {
          "datasetId": 40206

/Users/matthewharris/opt/miniconda3/envs/climate-bot/lib/python3.11/site-packages/pydantic/main.py:1111: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)




> Entering new AgentExecutor chain...

Invoking: `get_pf_info` with `{'query': 'California'}`



        mutation {
            getDatasetStatistics(input: { 
        country: "United States"
        address: "California"
                         warmingScenario: "1.5" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    
{
  "data": {
    "getDatasetStatistics": {
      "datasetStatisticsResponses": [
        {
          "datasetId": 40107,
          "midValue": "2.0",
          "name": "Days above 45\u00b0C (113\u00b0F)",
          "unit": "days",
          "warmingScenario": "1.5",
          "latitude": 37,
          "longitude": -119.6,
          "info": {}
        },
        {
          "datasetI

/Users/matthewharris/opt/miniconda3/envs/climate-bot/lib/python3.11/site-packages/pydantic/main.py:1111: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)




> Entering new AgentExecutor chain...

Invoking: `get_pf_info` with `{'query': 'Arizona'}`



        mutation {
            getDatasetStatistics(input: { 
        country: "United States"
        address: "Arizona"
                         warmingScenario: "1.5" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    
{
  "data": {
    "getDatasetStatistics": {
      "datasetStatisticsResponses": [
        {
          "datasetId": 40107,
          "midValue": "0.0",
          "name": "Days above 45\u00b0C (113\u00b0F)",
          "unit": "days",
          "warmingScenario": "1.5",
          "latitude": 34.2,
          "longitude": -111.4,
          "info": {}
        },
        {
          "datasetId": 

/Users/matthewharris/opt/miniconda3/envs/climate-bot/lib/python3.11/site-packages/pydantic/main.py:1111: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


query  \
0                                                      How will climate change in Miami   
1                              I live in California, how will climate change affect me?   
2  My home doesn't have its own water supply, how can I prepare for drought in Arizona?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

Now let's upload the dataset to Langsmith ...


In [286]:
dataset_name = "Climate Bot Evaluation  Set"
dataset_description = "Simple test set for evaluating the Climate Bot model"

# if not create a new one with the generated query examples
dataset = client.create_dataset(
    dataset_name=dataset_name, description=dataset_description
)
for e in eval_data:
    client.create_example(
        inputs={"question": e["query"]},
        outputs={
            "answer": e["ground_truths"],
            "context": e["context"],
        },
        dataset_id=dataset.id,
    )

print("Created a new dataset: ", dataset.name)

Created a new dataset:  Climate Bot Evaluation  Set


Now let's run a simple evaluation ...

In [299]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate
from langsmith.schemas import Run, Example

def run_chain(example: dict):
    """Use this for answer evaluation"""
    response = full_chain.invoke({
        "input": example["question"]
    })
    answer = response["fema_rag_chain_result"]["output"]
    docs = response["fema_rag_chain_result"]["context"]
    context = [doc.page_content for doc in docs]
    docs = [doc.metadata for doc in docs]
    return {"answer": answer, "context": context}


# Evaluator https://docs.smith.langchain.com/old/evaluation/faq/evaluator-implementations
eval_llm = ChatOpenAI(model="gpt-4o")
qa_evalulator = LangChainStringEvaluator(
        "cot_qa",
        prepare_data=lambda run, example: {
            "prediction": run.outputs["answer"],
            "reference": example.outputs["answer"],
            "input": example.inputs["question"],
        },
        config={"llm": eval_llm}  
    )

# Define evaluators
def retrieval_evaluator(run: Run, example: Example) -> dict:
    prediction = run.outputs.get("context") or []
    required = example.outputs.get("context") or []

    # Here we add code to compare chain retrieval results with test set results
    # This could be made more sophisticated by comparing the actual content of the documents
    score = 0
    for p in prediction:
        for r in required:
            if p in r:
                score += 1
                break

    score = score / len(required) if len(required) > 0 else 0

    return {"key":" retrieval_evaluator", "score": score}

experiment_results = evaluate(
    run_chain,
    data=dataset_name,
    evaluators=[qa_evalulator, retrieval_evaluator],
    experiment_prefix="rag-qa-oai",
    metadata={"variant": f"Quick test 1"},
)

View the evaluation results for experiment: 'rag-qa-oai-6d57d968' at:
https://smith.langchain.com/o/c25b80aa-0e39-544a-8c93-8567c04637a7/datasets/7db77fc3-0681-4fbe-91e2-df814a2e9497/compare?selectedSessions=f9533411-1b8f-41cc-b8f8-ba6ee3703f73




0it [00:00, ?it/s]



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

Invoking: `get_pf_info` with `{'query': 'Arizona'}`



        mutation {
            getDatasetStatistics(input: { 
        country: "United States"
        address: "Arizona"
                         warmingScenario: "1.5" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    

Invoking: `get_pf_info` with `{'query': 'Miami'}`



        mutation {
            getDatasetStatistics(input: { 
        country: "United States"
        address: "Miami"
                         warmingScenario: "1.5" 
                }) {
                datasetStatisticsResponses{
                    data

/Users/matthewharris/opt/miniconda3/envs/climate-bot/lib/python3.11/site-packages/pydantic/main.py:1111: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/matthewharris/opt/miniconda3/envs/climate-bot/lib/python3.11/site-packages/pydantic/main.py:1111: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


Based on the climate data for Arizona, there is a significant likelihood of drought conditions due to climate change. Specifically:

- The likelihood of a year-plus drought is 42%.
- The likelihood of a year-plus extreme drought is 17%.
- There is a decrease in total annual precipitation by 68 mm.
- There is an increase in dry hot days by 16 days.

Given these conditions, here are some steps you can take to prepare for drought in Arizona:

1. **Water Storage**:
   - Invest in water storage solutions such as rain barrels or large water tanks to collect and store water during wetter periods.
   - Ensure you have a sufficient supply of bottled water for drinking and cooking.

2. **Water Conservation**:
   - Implement water-saving fixtures and appliances in your home, such as low-flow showerheads, faucets, and toilets.
   - Fix any leaks promptly to prevent water wastage.

3. **Drought-Resistant Landscaping**:
   - Opt for xeriscaping, which involves using drought-resistant plants and mini

/Users/matthewharris/opt/miniconda3/envs/climate-bot/lib/python3.11/site-packages/pydantic/main.py:1111: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
3it [00:24,  8.14s/it]
